In [1]:
import sys
sys.path.append("../../utils/")
import time
from tqdm.notebook import tqdm
import gc
import glob
import copy
import pickle
import torch
import numpy as np
import pandas as pd
from torch.utils import tensorboard
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from loss import partial_correlation_score_torch_faster,correl_loss
from trainer import Multi_Trainer
import scipy

In [2]:
%%time

raw_path = "../../data_preprocessing/"
# get the indexs
with open("../../data_preprocessing/multi_in_ori_raw_idx.pkl","rb") as f:
    multi_in_ori_raw_idx = pickle.load(f)
with open("../../data_preprocessing/multi_in_ori_ori_idx.pkl","rb") as f:
    multi_in_ori_ori_idx = pickle.load(f)

train_inputs = np.load(raw_path+"new_multi_train_tsvd.npz")["arr_0"]  # tsvd
train_inputs = train_inputs[multi_in_ori_raw_idx]
train_targets = scipy.sparse.load_npz("../../../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_values.sparse.npz")
train_targets = train_targets[multi_in_ori_ori_idx]
train_targets = train_targets.toarray()
train_targets -= train_targets.mean(axis=1, keepdims=True)
train_targets /= train_targets.std(axis=1, ddof=1, keepdims=True)
with open(raw_path+"new_multi_tsvd_1024.pkl","rb") as f:
    pca2 = pickle.load(f)

components = torch.tensor(pca2.components_,device = "cuda")
def multi_correl_loss(pred, tgt):
    pred_ = pred @ components
    return correl_loss(pred_, tgt)
train_inputs.shape,train_targets.shape

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TruncatedSVD from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


CPU times: total: 9.53 s
Wall time: 35.5 s


((105868, 512), (105868, 23418))

In [3]:
config = dict(
    output_num = 1024,
    input_num = train_inputs.shape[1],
    dropout = 0.1,
    layers = 9,

    patience = 5,
    max_epochs = 100,
    criterion = multi_correl_loss,
    batch_size = 100,
    mlp_dims = [train_inputs.shape[1]*2,train_inputs.shape[1]],

    min_epoch = 20,
    n_folds = 3,
    folds_to_train = [0,1,2], # ,5,6,7,8
    kfold_random_state = 42,

    tb_dir = "./log/",

    optimizerparams = dict(lr=1e-4, weight_decay=1e-2,amsgrad = True),
    milestones = [20,25,30,35,40],
)


In [8]:
train_index = np.load(f"../../../input/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_idxcol.npz",allow_pickle=True)
meta = pd.read_csv("../../../input/open-problems-multimodal/metadata.csv",index_col = "cell_id")
meta = meta[meta.technology=="multiome"]
lbe = LabelEncoder()
meta["cell_type"] = lbe.fit_transform(meta["cell_type"])
meta["gender"] = meta.apply(lambda x:0 if x["donor"]==13176 else 1,axis =1)
meta_train = meta.reindex(train_index["index"])
meta_train = meta_train.iloc[multi_in_ori_ori_idx]

train_meta = meta_train["gender"].values.reshape(-1, 1)
train_inputs = np.concatenate([train_inputs,train_meta],axis= -1)
train_meta = meta_train["cell_type"].values.reshape(-1, 1)
train_inputs = np.concatenate([train_inputs,train_meta],axis= -1)
train_inputs.shape

(105868, 514)

In [9]:
class cell(torch.nn.Module):
    def __init__(self,input_dim,out_dim,dropout=0.1):
        super().__init__()
        self.weight_1 = torch.nn.Sequential(
            torch.nn.utils.weight_norm(torch.nn.Linear(input_dim,input_dim)),
            # torch.nn.Mish(),
            torch.nn.Softmax(dim= -1),
        )
        self.linear_0 = torch.nn.utils.weight_norm(torch.nn.Linear(input_dim,input_dim))
        self.linear_1 = torch.nn.Sequential(
            torch.nn.utils.weight_norm(torch.nn.Linear(input_dim,input_dim)),
            torch.nn.Mish(),
        )

        self.bn_1 = torch.nn.LayerNorm((input_dim))
        self.bn_2 = torch.nn.LayerNorm((out_dim))

        self.linear_2 = torch.nn.Sequential(
            torch.nn.utils.weight_norm(torch.nn.Linear(input_dim,out_dim)),
            torch.nn.Dropout(dropout),
            # torch.nn.Mish(),
            torch.nn.utils.weight_norm(torch.nn.Linear(out_dim,out_dim)),
            torch.nn.Dropout(dropout),
            torch.nn.Mish(),
        )

    def forward(self,x):
        x = self.bn_1(x)
        x_1 = self.linear_1(self.linear_0(x) * self.weight_1(x))
        x = self.bn_2(x_1+x)
        x = self.linear_2(x)
        return x

In [11]:
class modules(torch.nn.Module):
    def __init__(self,config):
        super().__init__()
        output_num = config["output_num"]
        input_num = config["input_num"]
        dropout = config["dropout"]
        mlp_dims = config["mlp_dims"]
        self.model = torch.nn.ModuleList()

        in_dim_array = [
            [mlp_dims[1],mlp_dims[1],mlp_dims[1]],
            [mlp_dims[1],mlp_dims[1],mlp_dims[1]],
            [mlp_dims[1],mlp_dims[1],mlp_dims[1]],
            # [mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1]],
        ]
        self.out_dim_array = [
            [mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1]],
            [mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1]],
            [mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1]],
            [mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1],mlp_dims[1]],
        ]

        for i in range(len(in_dim_array)): # 行
            temp_model = torch.nn.ModuleList()
            for j in range(len(in_dim_array[0])): # 列
                dim_in = in_dim_array[i][j]
                dim_out = self.out_dim_array[i][j]
                temp_model.append(
                    torch.nn.Sequential(
                        cell(dim_in,dim_out,dropout),
                        # cell(dim_in,dim_out,dropout),
                    )
                )
            self.model.append(temp_model)

In [12]:
class MLP(torch.nn.Module):
    def __init__(self,config):
        super().__init__()
        output_num = config["output_num"]
        self.input_num = config["input_num"]
        dropout = config["dropout"]
        mlp_dims = config["mlp_dims"]

        self.backbone = torch.nn.Linear(self.input_num ,self.input_num)
        self.embedding_1 = torch.nn.Embedding(2,self.input_num)
        self.embedding_2 = torch.nn.Embedding(7,self.input_num)

        self.model = modules(config)

        tail_input_dim = np.sum(np.array(self.model.out_dim_array)[-3:,-3:])
        self.tail = torch.nn.Sequential(
            torch.nn.Dropout(dropout),
            torch.nn.Linear(tail_input_dim,mlp_dims[1]*8),
            torch.nn.Mish(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(mlp_dims[1]*8,mlp_dims[1]*6),
            torch.nn.Mish(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(mlp_dims[1]*6,mlp_dims[1]*4),
            torch.nn.Mish(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(mlp_dims[1]*4,mlp_dims[1]*2),
            torch.nn.Mish(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(mlp_dims[1]*2,output_num),
        )
        

    def forward(self,xin):
        linears_1_index = 0  # 列
        linears_2_index = 0  # 行
        # xin_1 = self.projection(xin[:,:self.input_num])
        xin = self.backbone(xin[:,:self.input_num])  + self.embedding_1(xin[:,-2].int())  # + self.embedding_1(xin[:,-2].int())# 
        # neck
        temp_model_list = self.model.model[0]
        res_array = []
        res_list  = []
        temp_out = xin
        linears_1_index += 1
        for id,i in enumerate(temp_model_list):
            if id ==0 :
                temp_out = i(
                    temp_out+xin
                    )
                linears_2_index += 1
            else:
                temp_out = i(
                    temp_out+xin
                    )
                linears_2_index += 1
            res_list.append(temp_out) 
        res_array += res_list[-3:]

        temp_model_list = self.model.model[1]
        temp_out = xin
        linears_1_index += 1
        for i in range(len(temp_model_list)):
            if i == -1:
                temp_out = temp_model_list[i](
                  res_list[i]
                )
            else:
                temp_out = temp_model_list[i](
                res_list[i]+temp_out
                    )
            res_list[i] = temp_out
        res_array += res_list[-3:]

        temp_model_list = self.model.model[2]
        temp_out = xin
        linears_1_index += 1
        for i in range(len(temp_model_list)):
            if i == -1:
                temp_out = temp_model_list[i](
                   res_list[i]
                )
            else:
                temp_out = temp_model_list[i](
                    res_list[i]+temp_out
                )
            res_list[i] = temp_out
        res_array += res_list[-3:]


        # temp_model_list = self.model.model[3]
        # temp_out = xin
        # linears_1_index += 1
        # for i in range(len(temp_model_list)):
        #     if i == -1:
        #         temp_out = temp_model_list[i](
        #            res_list[i]
        #         )
        #     else:
        #         temp_out = temp_model_list[i](
        #             res_list[i]+temp_out
        #             )
 
        #     res_list[i] = temp_out
        # res_array += res_list[-3:]

        res_array = torch.concat(res_array,dim = -1)
        res_array = self.tail(res_array)
        del res_list,temp_out,xin,temp_model_list
        torch.cuda.empty_cache()
        # tail
        return res_array


In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f"machine has {torch.cuda.device_count()} cuda devices")
    print(f"model of first cuda device is {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")

trainer = Multi_Trainer(device,config)
kfold = KFold(n_splits=config["n_folds"],shuffle=True, random_state=config["kfold_random_state"]) #
FOLDS_LIST = list(kfold.split(range(train_inputs.shape[0]))) #,groups= meta_train.donor
print("Training started")
fold_scores = []
for num_fold in config["folds_to_train"]:
    model = MLP(config)
    best_score = trainer.train_one_fold(num_fold,FOLDS_LIST,train_inputs,train_targets,model,components)
    fold_scores.append(best_score)
print("\n")
print(f"Final average score is {sum(fold_scores)/len(fold_scores)}")

machine has 1 cuda devices
model of first cuda device is NVIDIA GeForce RTX 3070
Training started
Adjusting learning rate of group 0 to 1.0000e-04.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/706 [00:00<?, ?it/s]

  0%|          | 0/353 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-04.
epoch-0 train_loss:-0.6417288199362606 val_loss:-0.6663166078562102 corr_score:0.6663288474082947


...

  0%|          | 0/353 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-08.
epoch-34 train_loss:-0.6734802851258189 val_loss:-0.6730890030901425 corr_score:0.6731016039848328


  0%|          | 0/706 [00:00<?, ?it/s]

  0%|          | 0/353 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-08.
epoch-35 train_loss:-0.6734820141670724 val_loss:-0.6730890030901425 corr_score:0.6731017231941223
out of patience


Final average score is 0.6726819276809692


In [27]:
np.mean(fold_scores)

0.6725666666666666

In [ ]:
# [tensor(0.6723),tensor(0.6723), tensor(0.6731)]

[tensor(0.6720), tensor(0.6719), tensor(0.6727)]

In [15]:
fold_scores # [0.6723,0.6723,0.6731]

In [16]:
class Tester:
    def __init__(self,device,config):
        self.device = device
        self.config = config
      
    def std(self,x):
        return (x - np.mean(x,axis=1).reshape(-1,1)) / np.std(x,axis=1).reshape(-1,1)

    def test_fn_ensemble(self,model_list, dl_test):
        
        res = np.zeros(
            (self.len, 23418), )
        
        for model in model_list:
            model.eval()
            
        cur = 0
        for inpt in tqdm(dl_test):
            inpt = inpt[0]
            
            with torch.no_grad():
                pred_list = []
                inpt = inpt.to(self.device)
                # print("inpt",inpt.shape)
                for id,model in enumerate(model_list):
                    model.to(self.device)
                    model.eval()
                    pred = model(inpt)
                    model.to("cpu")
                    # print("pred",pred.shape)
                    pred = self.std(pred.cpu().numpy()@pca2.components_) * self.weight[id]
                    pred_list.append(pred)
                pred = sum(pred_list)/len(pred_list)
                
            # print(res.shape, cur, cur+pred.shape[0], res[cur:cur+pred.shape[0]].shape, pred.shape)
            res[cur:cur+pred.shape[0]] = pred
            cur += pred.shape[0]
                
        return {"preds":res}

    def load_model(self,path ):
        model_list = []
        for fn in tqdm(glob.glob(path)):
            prefix = fn[:-len("_best_params.pth")]
            config_fn = prefix + "_config.pkl"
            
            config = pickle.load(open(config_fn, "rb"))

            model = MLP(config)
            model.to("cpu")
            
            params = torch.load(fn)
            model.load_state_dict(params)
            
            model_list.append(model)
        print("model loaded")
        return model_list
    
    def load_data(self,ds_test ):
        self.len = ds_test.shape[0]
        return ds_test

    def test(self,data_path,model_path = "./*_best_params.pth",weight = fold_scores):
        self.weight = weight
        model_list = self.load_model(model_path)
        test_inputs = self.load_data(data_path)
        ds_test = torch.utils.data.TensorDataset(test_inputs)
        gc.collect()
        dl_test = torch.utils.data.DataLoader(ds_test, batch_size=1024, shuffle=False, drop_last=False)
        test_pred = self.test_fn_ensemble(model_list, dl_test)["preds"]
        del model_list
        del dl_test
        del test_inputs
        gc.collect()
        print(test_pred.shape)
        np.save("test_pred.npy",test_pred)
        return test_pred
        

In [15]:
test = np.load(raw_path+"new_multi_test_tsvd.npz")["arr_0"]  # tsvd
test_index = np.load(f"../../../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz",allow_pickle=True)

meta_test= meta.reindex(test_index["index"])
test_meta = meta_test["gender"].values.reshape(-1, 1)
test = np.concatenate([test,test_meta],axis= -1)
test_meta = meta_test["cell_type"].values.reshape(-1, 1)
test = np.concatenate([test,test_meta],axis= -1)
test = torch.tensor(test,dtype=torch.float)
test.shape

torch.Size([55935, 514])

In [23]:

tester = Tester( torch.device("cuda"),config)
test_pred = tester.test(test)

  0%|          | 0/3 [00:00<?, ?it/s]

model loaded


  0%|          | 0/55 [00:00<?, ?it/s]

(55935, 23418)


In [24]:
import numpy as np
import pandas as pd
import gc
# test_pred = np.load("test_pred.npy")
test_pred.shape

(55935, 23418)

In [25]:
def submit(test_pred,cite_path = r"C:\Users\Olive\Downloads\Compressed\submission_best.zip"):
    
    # Read the table of rows and columns required for submission
    eval_ids = pd.read_parquet("../data/others/evaluation.parquet")

    # Convert the string columns to more efficient categorical types
    #eval_ids.cell_id = eval_ids.cell_id.apply(lambda s: int(s, base=16))

    eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
    eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

    submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
    print("data loaded")
    y_columns = np.load("../data/others/train_multi_targets_idxcol.npz",allow_pickle=True)["columns"]
    test_index = np.load("../data/others/test_multi_inputs_idxcol.npz",allow_pickle=True)["index"]

    cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
    assert len(cell_dict)  == len(test_index)

    gene_dict = dict((k,v) for v,k in enumerate(y_columns))
    assert len(gene_dict) == len(y_columns)

    eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
    eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

    valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)
    valid_multi_rows = valid_multi_rows.to_numpy()
    eval_ids_gene_num[valid_multi_rows].to_numpy()
    
    submission.iloc[valid_multi_rows] = test_pred[eval_ids_cell_num[valid_multi_rows].to_numpy(),
    eval_ids_gene_num[valid_multi_rows].to_numpy()]

    del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
    gc.collect()

    submission.reset_index(drop=True, inplace=True)
    submission.index.name = 'row_id'

    # Merging in the CITEseq submission
    cite_submission = pd.read_csv(cite_path)
    cite_submission = cite_submission.set_index("row_id")
    cite_submission = cite_submission["target"]

    submission[submission.isnull()] = cite_submission[submission.isnull()]
    submission.isnull().any()
    print("start -> submission.zip")
    submission.to_csv("submission.zip")

    print("submission.zip saved!")


In [26]:
submit(test_pred,cite_path=r"D:\python_project\MSCI\model_ensemble\submission_best.zip")

data loaded
start -> submission.zip
submission.zip saved!
